https://www.kaggle.com/fernandol/countries-of-the-world/data

In [1]:
# Import statements! Are loans given to people who actually need it, 3 more files, contain poverty index of every country
from functools import reduce
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, Imputer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

In [2]:
loans = pd.read_csv("data-science-for-good-kiva-crowdfunding/kiva_loans.csv")
mpi_locations = pd.read_csv("data-science-for-good-kiva-crowdfunding/kiva_mpi_region_locations.csv")
world_countries = pd.read_csv("data-science-for-good-kiva-crowdfunding/countries of the world.csv", thousands = '.')

# First let's do all the data work with world_countries data set

In [3]:
world_countries["Country"] = world_countries["Country"].str.strip()

In [4]:
world_countries.loc[world_countries["Country"] == 'Albania']

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"


In [ ]:
world_countries = world_countries.fillna('0') # It's not the best solution but we must do this to work with the comma separated values

In [ ]:
for k in range(0, len(world_countries)):
    
    world_countries["Coastline (coast/area ratio)"][k] = world_countries["Coastline (coast/area ratio)"][k].replace(',','.')
    world_countries["Net migration"][k] = world_countries["Net migration"][k].replace(',','.')
    world_countries["Infant mortality (per 1000 births)"][k] = world_countries["Infant mortality (per 1000 births)"][k].replace(',','.')
    world_countries["Literacy (%)"][k] = world_countries["Literacy (%)"][k].replace(',','.')
    world_countries["Phones (per 1000)"][k] = world_countries["Phones (per 1000)"][k].replace(',','.')
    world_countries["Arable (%)"][k] = world_countries["Arable (%)"][k].replace(',','.')
    world_countries["Crops (%)"][k] = world_countries["Crops (%)"][k].replace(',','.')
    world_countries["Other (%)"][k] = world_countries["Other (%)"][k].replace(',','.')
    world_countries["Birthrate"][k] = world_countries["Birthrate"][k].replace(',','.')
    world_countries["Deathrate"][k] = world_countries["Deathrate"][k].replace(',','.')
    world_countries["Agriculture"][k] = world_countries["Agriculture"][k].replace(',','.')
    world_countries["Industry"][k] = world_countries["Industry"][k].replace(',','.')
    world_countries["Service"][k] = world_countries["Service"][k].replace(',','.')
    world_countries["Pop. Density (per sq. mi.)"][k] = world_countries["Pop. Density (per sq. mi.)"][k].replace(',','.')



/home/shelob/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/shelob/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/shelob/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/h

In [ ]:
world_countries_revise = pd.DataFrame.copy(world_countries)

In [ ]:
world_countries_revise["Coastline (coast/area ratio)"] = pd.to_numeric(world_countries["Coastline (coast/area ratio)"])
world_countries_revise["Infant mortality (per 1000 births)"] = pd.to_numeric(world_countries["Infant mortality (per 1000 births)"])
world_countries_revise["Literacy (%)"] = pd.to_numeric(world_countries["Literacy (%)"])
world_countries_revise["Phones (per 1000)"] = pd.to_numeric(world_countries["Phones (per 1000)"])
world_countries_revise["Arable (%)"] = pd.to_numeric(world_countries["Arable (%)"])
world_countries_revise["Crops (%)"] = pd.to_numeric(world_countries["Crops (%)"])
world_countries_revise["Other (%)"] = pd.to_numeric(world_countries["Other (%)"])
world_countries_revise["Net migration"] = pd.to_numeric(world_countries["Net migration"])
world_countries_revise["Birthrate"] = pd.to_numeric(world_countries["Birthrate"])
world_countries_revise["Deathrate"] = pd.to_numeric(world_countries["Deathrate"])
world_countries_revise["Agriculture"] = pd.to_numeric(world_countries["Agriculture"])
world_countries_revise["Industry"] = pd.to_numeric(world_countries["Industry"])
world_countries_revise["Service"] = pd.to_numeric(world_countries["Service"])
world_countries_revise["GDP ($ per capita)"] = pd.to_numeric(world_countries["GDP ($ per capita)"])
world_countries_revise["Pop. Density (per sq. mi.)"] = pd.to_numeric(world_countries["Pop. Density (per sq. mi.)"])

In [ ]:
mpi_locations.head(5)

In [ ]:
loans.head(1)

In [ ]:
world_countries.head(1)

In [ ]:
loans_no_nan = loans[pd.notnull(loans['region'])]

In [ ]:
loans_merge = pd.merge(loans_no_nan, mpi_locations, on = 'region')

In [ ]:
loans_merge.plot(kind = "scatter", x="lon", y="lat")
plt.show()

In [ ]:
loans_merge.plot(kind = "scatter", x="lon", y="lat", alpha = .5,
                  s = loans_merge["MPI"] * 1000 , label = "MPI", figsize = (20, 14),
                  c = "loan_amount", cmap = plt.get_cmap("jet"), colorbar = True,
                  )
plt.legend()
plt.show()

In [ ]:
copy = pd.DataFrame.copy(loans_merge)

In [ ]:
copy = copy.drop("geo", 1)
copy = copy.drop("country_code", 1)
copy = copy.drop("use", 1)
copy = copy.drop("partner_id", 1)
copy = copy.drop("disbursed_time", 1)
copy = copy.drop("funded_time", 1)
copy = copy.drop("date", 1)
copy = copy.drop("tags", 1)
copy = copy.drop("ISO", 1)
copy = copy.drop("country_y", 1)
copy = copy.drop("world_region", 1)
copy = copy.drop("posted_time", 1)
copy = copy.drop("id", 1)
copy = copy.drop("region", 1)
copy = copy.drop("currency", 1)
copy = copy.drop("LocationName", 1)
copy = copy.drop("lat", 1)
copy = copy.drop("lon", 1)

In [ ]:
copy.info()

In [ ]:
copy.head()

In [ ]:
copy.info()

In [ ]:
encoder = LabelEncoder()

In [ ]:
sector_cat = copy["sector"]
activity_cat = copy["activity"]
country_cat = copy["country_x"]
# region_cat = copy_2["region"]
gender_cat = copy["borrower_genders"]
repay_cat = copy["repayment_interval"]

In [ ]:
# region_cat = region_cat.fillna('Other Region')
gender_cat = gender_cat.fillna('Other Gender')

In [ ]:
sector_encode = encoder.fit_transform(sector_cat)
activity_encode = encoder.fit_transform(activity_cat)
# region_encode = encoder.fit_transform(region_cat)
gender_encode = encoder.fit_transform(gender_cat)
repay_encode = encoder.fit_transform(repay_cat)
# climate_encode = encoder.fit_transform(climate_cat)
country_encode = encoder.fit_transform(country_cat)

In [ ]:
copy["sector"] = sector_encode
copy["activity"] = activity_encode
# copy["region"] = region_encode
copy["borrower_genders"] = gender_encode
copy["repayment_interval"] = repay_encode
# copy["Climate"] = climate_encode
copy["country_x"] = country_encode

In [ ]:
imputer = Imputer(strategy = "median")

In [ ]:
imputer.fit(copy)
X = imputer.transform(copy)
transform_of_copy = pd.DataFrame(X, columns = copy.columns)
transform_of_copy

In [ ]:
train_set_copy, test_set_copy = train_test_split(transform_of_copy, test_size = 0.2, random_state = 42)

In [ ]:
y_train = train_set_copy["country_x"]
y_test = test_set_copy["country_x"]
x_test = test_set_copy.drop("country_x", 1)
x_train = train_set_copy.drop("country_x", 1)

In [ ]:
dt_clf = DecisionTreeClassifier(random_state = 42)

In [ ]:
dt_clf.fit(x_train, y_train)
dt_clf

In [ ]:
dt_clf.classes_

In [ ]:
cross_val_score(dt_clf, x_train, y_train, cv = 3, scoring = "accuracy")

In [ ]:
y_train_predict_dt = cross_val_predict(dt_clf, x_train, y_train, cv = 3)
y_train_predict_dt

In [ ]:
confusion_dt = confusion_matrix(y_train, y_train_predict_dt)
confusion_dt

In [ ]:
plt.matshow(confusion_dt, cmap = plt.cm.gray)
plt.show()

In [ ]:
sum_of_row = confusion_dt.sum(axis = 1, keepdims = True)
norm_conf_dt = confusion_dt / sum_of_row

In [ ]:
np.fill_diagonal(norm_conf_dt, 0)
plt.matshow(norm_conf_dt, cmap = plt.cm.gray)
plt.show()

In [ ]:
precision_score(y_train, y_train_predict_dt, average = 'weighted')

In [ ]:
recall_score(y_train, y_train_predict_dt, average = 'weighted')

In [ ]:
f1_score(y_train, y_train_predict_dt, average = 'weighted')

In [ ]:
dt_predictions = dt_clf.predict(x_test) #predict how the classes from our test set
dt_predictions

In [ ]:
print(accuracy_score(y_test, dt_predictions)) #88.5% accurate on our test set!!!!

# Random Forest Comparison

In [ ]:
random_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)

In [ ]:
random_clf.fit(x_train, y_train)
random_clf

In [ ]:
random_clf.classes_

In [ ]:
cross_val_score(random_clf, x_train, y_train, cv = 3, scoring = "accuracy")

In [ ]:
y_train_predict_random = cross_val_predict(random_clf, x_train, y_train, cv = 3)
y_train_predict_random

In [ ]:
confusion_random = confusion_matrix(y_train, y_train_predict_random)
confusion_random

In [ ]:
plt.matshow(confusion_random, cmap = plt.cm.gray)
plt.show()

In [ ]:
sum_of_row_rf = confusion_random.sum(axis = 1, keepdims = True)
norm_conf_rf = confusion_random / sum_of_row_rf

In [ ]:
np.fill_diagonal(norm_conf_rf, 0)
plt.matshow(norm_conf_rf, cmap = plt.cm.gray)
plt.show()

In [ ]:
precision_score(y_train, y_train_predict_random, average = 'weighted')

In [ ]:
recall_score(y_train, y_train_predict_random, average = 'weighted')

In [ ]:
f1_score(y_train, y_train_predict_random, average = 'weighted')

In [ ]:
random_predictions = random_clf.predict(x_test) #predict how the classes from our test set
random_predictions

In [ ]:
print(accuracy_score(y_test, random_predictions)) 

In [ ]:
random_clf.feature_importances_

In [ ]:
for name, score in zip(list(x_train), random_clf.feature_importances_):
    print(name, score) # remember these are percents, so multiply by 100

In [ ]:
# copy_2 = pd.merge(copy, world_countries_revise, left_on = "country_x", right_on = "Country")

In [ ]:
# copy_2.loc[copy_2["country_x"] == "El Salvador"]

In [ ]:
# copy_2.info()

In [ ]:
# copy_2["Climate"].value_counts() # We can encode this one

In [ ]:
# copy_2 = copy_2.drop("Country", 1)
# copy_2 = copy_2.drop("Region", 1)

In [ ]:
# copy_2 = copy_2.drop("Region", 1)

In [ ]:
# copy_2 = copy_2.drop("currency", 1)

In [ ]:
# copy_2 = copy_2.drop("LocationName", 1)

In [ ]:
# copy_2["GDP ($ per capita)"] = pd.to_numeric(copy_2["GDP ($ per capita)"])

In [ ]:
encoder = LabelEncoder()

In [ ]:
# sector_cat = copy_2["sector"]
# activity_cat = copy_2["activity"]
# country_cat = copy_2["country_x"]
# # region_cat = copy_2["region"]
# gender_cat = copy_2["borrower_genders"]
# repay_cat = copy_2["repayment_interval"]
# climate_cat = copy_2["Climate"]

In [ ]:
# region_cat = region_cat.fillna('Other Region')
# gender_cat = gender_cat.fillna('Other Gender')

In [ ]:
# sector_encode = encoder.fit_transform(sector_cat)
# activity_encode = encoder.fit_transform(activity_cat)
# region_encode = encoder.fit_transform(region_cat)
# gender_encode = encoder.fit_transform(gender_cat)
# repay_encode = encoder.fit_transform(repay_cat)
# climate_encode = encoder.fit_transform(climate_cat)
# country_encode = encoder.fit_transform(country_cat)

In [ ]:
# copy_2["sector"] = sector_encode
# copy_2["activity"] = activity_encode
# copy_2["region"] = region_encode
# copy_2["borrower_genders"] = gender_encode
# copy_2["repayment_interval"] = repay_encode
# copy_2["Climate"] = climate_encode
# copy_2["country_x"] = country_encode

In [ ]:
# copy_2.head()

In [ ]:
# imputer = Imputer(strategy = "median")

In [ ]:
# imputer.fit(copy_2)
# X = imputer.transform(copy_2)
# transform_world = pd.DataFrame(X, columns = copy_2.columns)
# transform_world

In [ ]:
# %matplotlib inline
# transform_world.hist(bins = 50, figsize = (20, 15))
# plt.show()

In [ ]:
# correlation_matrix = transform_world.corr()

In [ ]:
# correlation_matrix["sector"].sort_values(ascending = False)

In [ ]:
# correlation_matrix["country_x"].sort_values(ascending = False)

# Now let's experiement with some classification techniques!

In [ ]:
# train_set, test_set = train_test_split(transform_world, test_size = 0.2, random_state = 42)

In [ ]:
# y_train = train_set["sector"]
# y_test = test_set["sector"]
# x_test = test_set.drop("sector", 1)
# x_train = train_set.drop("sector", 1)

In [ ]:
# dt_clf = DecisionTreeClassifier(random_state = 42)

In [ ]:
# dt_clf.fit(x_train, y_train)
# dt_clf

In [ ]:
# dt_clf.classes_

In [ ]:
# cross_val_score(dt_clf, x_train, y_train, cv = 3, scoring = "accuracy")

In [ ]:
# y_train_predict_dt = cross_val_predict(dt_clf, x_train, y_train, cv = 3)
# y_train_predict_dt

In [ ]:
# confusion_dt = confusion_matrix(y_train, y_train_predict_dt)
# confusion_dt

In [ ]:
# plt.matshow(confusion_dt, cmap = plt.cm.gray)
# plt.show()

In [ ]:
# sum_of_row = confusion_dt.sum(axis = 1, keepdims = True)
# norm_conf_dt = confusion_dt / sum_of_row

In [ ]:
# np.fill_diagonal(norm_conf_dt, 0)
# plt.matshow(norm_conf_dt, cmap = plt.cm.gray)
# plt.show()

In [ ]:
# precision_score(y_train, y_train_predict_dt, average = 'weighted')

In [ ]:
# recall_score(y_train, y_train_predict_dt, average = 'weighted')

In [ ]:
# f1_score(y_train, y_train_predict_dt, average = 'weighted')

In [ ]:
# dt_predictions = dt_clf.predict(x_test) #predict how the classes from our test set
# dt_predictions

In [ ]:
# print(accuracy_score(y_test, dt_predictions)) #88.5% accurate on our test set!!!!

# Since the Decision Tree seems to be overfitting, lets try the Random Forest

In [ ]:
# random_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)

In [ ]:
# random_clf.fit(x_train, y_train)
# random_clf

In [ ]:
# random_clf.classes_

In [ ]:
# cross_val_score(random_clf, x_train, y_train, cv = 3, scoring = "accuracy")

In [ ]:
# y_train_predict_random = cross_val_predict(random_clf, x_train, y_train, cv = 3)
# y_train_predict_random

In [ ]:
# confusion_random = confusion_matrix(y_train, y_train_predict_random)
# confusion_random

In [ ]:
# plt.matshow(confusion_random, cmap = plt.cm.gray)
# plt.show()

In [ ]:
# sum_of_row_rf = confusion_random.sum(axis = 1, keepdims = True)
# norm_conf_rf = confusion_random / sum_of_row_rf

In [ ]:
# np.fill_diagonal(norm_conf_rf, 0)
# plt.matshow(norm_conf_rf, cmap = plt.cm.gray)
# plt.show()

In [ ]:
# precision_score(y_train, y_train_predict_random, average = 'weighted')

In [ ]:
# recall_score(y_train, y_train_predict_random, average = 'weighted')

In [ ]:
# f1_score(y_train, y_train_predict_random, average = 'weighted')

In [ ]:
# random_predictions = random_clf.predict(x_test) #predict how the classes from our test set
# random_predictions

In [ ]:
# print(accuracy_score(y_test, random_predictions)) 

In [ ]:
# random_clf.feature_importances_

In [ ]:
# for name, score in zip(list(x_train), random_clf.feature_importances_):
#     print(name, score) # remember these are percents, so multiply by 100

In [ ]:
# copy_2["country_x"].value_counts()